## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes

## Limpeza e pré processamento dos dados

### Carregamento

Apenas colunas que serão utilizadas na análise

In [ ]:
desired_cols = [
  "DT_NOTIFIC", # Início do caso 
  "DT_SIN_PRI",

  "ANO_NASC",  # Info pessoal
  "CS_SEXO",
  "CS_GESTANT",
  "CS_RACA",
  "CS_ESCOL_N",
  "SG_UF",
  "ID_MN_RESI",
  "ID_RG_RESI",

  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",

  "HOSPITALIZ", # Desfecho
  "CLASSI_FIN", 
  "EVOLUCAO",
  "DT_ENCERRA"
]

In [ ]:
# Carregando dataset
DATA_PATH = "./data/original-2/DENGBR23.csv"
df = pd.read_csv(DATA_PATH, low_memory=False, usecols=desired_cols)

In [ ]:
df.head()

In [ ]:
df.info()

### Filtragem

#### SG_UF

31 - Minas Gerais<br>
32 - Espírito Santo<br>
33 - Rio de Janeiro<br>
35 - São Paulo

In [ ]:
target_ufs = [31, 32, 33, 35]
df = df[df["SG_UF"].isin(target_ufs)]
df["SG_UF"].value_counts()

#### CLASSI_FIN

Somente casos confirmados<br>
10 - Dengue<br>
11 - Dengue com sinais de alarme<br>
12 - Dengue grave

In [ ]:
target_classi_fins = [10, 11, 12]
df = df[df["CLASSI_FIN"].isin(target_classi_fins)]
df["CLASSI_FIN"].unique()

#### ANO_NASC

In [ ]:
df["ANO_NASC"].unique()

In [ ]:
df = df.dropna(subset=["ANO_NASC"])
df["ANO_NASC"].isna().sum()

#### CS_SEXO

In [ ]:
df["CS_SEXO"].unique()

In [ ]:
df["CS_SEXO"] = df["CS_SEXO"].replace("I", np.nan)
df = df.dropna(subset=["CS_SEXO"])
df["CS_SEXO"].unique()

#### CS_GESTANT

1 - 1º Trimestre <br>
2 - 2º Trimestre <br>
3 - 3º Trimestre <br>
4 - Idade gestacional ignorada <br>
5 - Não <br>
6 - Não se aplica <br>
9 - Ignorado <br>

In [ ]:
df["CS_GESTANT"].unique()

In [ ]:
df["CS_GESTANT"] = df["CS_GESTANT"].replace(9.0, np.nan)
df = df.dropna(subset=["CS_GESTANT"])
df["CS_GESTANT"].unique()

#### CS_RACA

1- branca <br>
2- preta <br>
3- amarela <br>
4- parda <br>
5- indígena <br>
9- Ignorado <br>

In [ ]:
df["CS_RACA"].unique()

In [ ]:
df["CS_RACA"] = df["CS_RACA"].replace(9.0, np.nan)
df = df.dropna(subset=["CS_RACA"])
df["CS_RACA"].unique()

#### CS_ESCOL_N

Ign/Branco
,9 ,09,99 <br>
Analfabeto
0 ,00 <br>
1ª a 4ª série incompleta do EF
1
,01 <br>
4ª série completa do EF
2
,02 <br>
5ª a 8ª série incompleta do EF 3 ,03 <br>
Ensino fundamental completo 4 ,04 <br>
Ensino médio incompleto
5
,05 <br>
Ensino médio completo
6
,06 <br>
Educação superior incompleta
7
,07 <br>
Educação superior completa
8 ,08 <br>
Não se aplica
10

In [ ]:
df["CS_ESCOL_N"].unique()

In [ ]:
df["CS_ESCOL_N"] = df["CS_ESCOL_N"].replace(9.0, np.nan).replace(10.0, np.nan)
df = df.dropna(subset=["CS_ESCOL_N"])
df["CS_ESCOL_N"] = df["CS_ESCOL_N"].astype(int)
df["CS_ESCOL_N"].unique()

In [ ]:
df["SG_UF"].unique()

#### Sintomas e comorbidades

In [ ]:
df = df.fillna(2)
df.info()

### Transformação de dados

#### ANO_NASC

In [ ]:
current_year = 2025
df["IDADE"] = current_year - df["ANO_NASC"]
df["IDADE"] = df["IDADE"].astype(int)
df["IDADE"].unique()

In [ ]:
df["FAIXA_IDADE"] = pd.cut(df["IDADE"], bins=[0, 10, 20, 30, 40, 60, 100], labels=["0-10", "11-20", "21-30", "31-40", "41-60", "60+"])
df = df.dropna(subset=["FAIXA_IDADE"])
df_encoded_age = pd.get_dummies(df, columns=["FAIXA_IDADE"])

#### CS_SEXO

In [ ]:
df_encoded_sex = pd.get_dummies(df_encoded_age, columns=["CS_SEXO"], drop_first=True)
df_encoded_sex["CS_SEXO_M"] = df_encoded_sex["CS_SEXO_M"].astype(int)

#### CS_GESTANT

1, 2, 3, 4 -> Sim <br>
5, 6 -> Não

In [ ]:
df_encoded_sex["CS_GESTANT"] = df_encoded_sex["CS_GESTANT"].replace([1, 2, 3, 4], "S")
df_encoded_sex["CS_GESTANT"] = df_encoded_sex["CS_GESTANT"].replace([5, 6], "N")
df_encoded_gest = pd.get_dummies(df_encoded_sex, columns=["CS_GESTANT"], drop_first=True)
df_encoded_gest["CS_GESTANT_S"] = df_encoded_gest["CS_GESTANT_S"].astype(int) 
df_encoded_gest.head()

#### CS_RACA

In [ ]:
# Trocando raça de float pra int
df_encoded_gest["CS_RACA"] = df_encoded_gest["CS_RACA"].astype(int)

# One hot encoding
df_encoded_raca = pd.get_dummies(df_encoded_gest, columns=["CS_RACA"])

df_encoded_raca.head()

#### CS_ESCOL_N

In [ ]:
df_encoded_escol = pd.get_dummies(df_encoded_raca, columns=["CS_ESCOL_N"])

# Trocando true e false para 1 e 0
colunas_bool = df_encoded_escol.select_dtypes(include='bool').columns
df_encoded_escol[colunas_bool] = df_encoded_escol[colunas_bool].astype(int)

df_encoded_escol.head()

#### Sintomas e Comorbidades

Aqui é necessário transformar o 2 em 0 

In [ ]:
sint_com_cols = [
  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",
]

In [ ]:
df_encoded_final = df_encoded_escol.copy()
df_encoded_final.head()

In [ ]:
df_encoded_final[sint_com_cols] = df_encoded_final[sint_com_cols].replace({2:0})
df_encoded_final[sint_com_cols] = df_encoded_final[sint_com_cols].astype(int)
df_encoded_final.head()

### Seleção de colunas para clusterização

In [ ]:
clustering_cols = [
  "CS_ESCOL_N_0", # Categoricos transformados
  "CS_ESCOL_N_1",
  "CS_ESCOL_N_2",
  "CS_ESCOL_N_3",
  "CS_ESCOL_N_4",
  "CS_ESCOL_N_5",
  "CS_ESCOL_N_6",
  "CS_ESCOL_N_7",
  "CS_ESCOL_N_8",

  "CS_SEXO_M",
  "CS_GESTANT_S",

  "CS_RACA_1",
  "CS_RACA_2",
  "CS_RACA_3",
  "CS_RACA_4",
  "CS_RACA_5",

  "FAIXA_IDADE_0-10",
  "FAIXA_IDADE_11-20",
  "FAIXA_IDADE_21-30",
  "FAIXA_IDADE_31-40",
  "FAIXA_IDADE_41-60",
  "FAIXA_IDADE_60+",
  
  "FEBRE",    # Sintomas
  "MIALGIA",
  "CEFALEIA",
  "EXANTEMA",
  "VOMITO",
  "NAUSEA",
  "DOR_COSTAS",
  "CONJUNTVIT",
  "ARTRITE",
  "ARTRALGIA",
  "PETEQUIA_N",
  "LEUCOPENIA",
  "LACO",
  "DOR_RETRO",

  "DIABETES",   # Comorbidades
  "HEMATOLOG",
  "HEPATOPAT",
  "RENAL",
  "HIPERTENSA",
  "ACIDO_PEPT",
  "AUTO_IMUNE",
]

df_clustering = df_encoded_final[clustering_cols]
df_clustering.head()

#### Não é preciso normalização para dados com one hot encoding

In [ ]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df_clustering)

## Clusterização

### KModes com One Hot Encoding

In [ ]:
# sample = df_clustering.sample(frac=0.2, random_state=1)

# costs = []
# K_range = range(1, 11)

# for k in K_range:
#     km = KModes(n_clusters=k, init='Huang', n_init=1, verbose=1)
#     km.fit_predict(sample)
#     costs.append(km.cost_)

# plt.plot(K_range, costs, marker='o')
# plt.xlabel('Número de clusters (K)')
# plt.ylabel('Custo (Distância intra-cluster)')
# plt.title('Método do Cotovelo para K-Modes')
# plt.grid()
# plt.show()

In [ ]:
# Crie o modelo com K=5 e uma inicialização robusta
km = KModes(n_clusters=3, init='Huang', n_init=3, verbose=1)

# Ajuste o modelo aos dados completos e obtenha os clusters
clusters = km.fit_predict(df_clustering)  # df = seu dataframe categórico

# Agora você pode adicionar os clusters no dataframe
df_clustering['cluster'] = clusters

In [ ]:
from sklearn.decomposition import TruncatedSVD

X = df_clustering.values  # pode ser matriz esparsa, ex: scipy.sparse.csr_matrix

svd = TruncatedSVD(n_components=2, random_state=1)
X_svd = svd.fit_transform(X)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_svd[:, 0], X_svd[:, 1], c=df_clustering['cluster'], cmap='tab10', s=10)
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Visualização dos clusters com TruncatedSVD")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
# TruncatedSVD para 3 componentes
svd = TruncatedSVD(n_components=3, random_state=1)
X_svd_3d = svd.fit_transform(X)

# Plot 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    X_svd_3d[:, 0], X_svd_3d[:, 1], X_svd_3d[:, 2],
    c=df_clustering['cluster'], cmap='tab10', s=15, alpha=0.8
)

ax.set_title("Visualização 3D dos clusters com TruncatedSVD")
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_zlabel("Componente 3")

legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
ax.add_artist(legend1)

plt.show()

In [ ]:
df["cluster"] = clusters
df.head()

In [ ]:
df["EVOLUCAO"].unique()

In [ ]:
df["HOSPITALIZ"].unique()

In [ ]:
# contagem de elementos por cluster
df["cluster"].value_counts()

In [ ]:
df.groupby('cluster')['IDADE'].mean().plot(kind='bar')
plt.title('Média de Idade por Cluster')
plt.ylabel('Idade média')
plt.show()

In [ ]:
import seaborn as sns

cluster_profile = df_clustering.groupby('cluster').mean()

plt.figure(figsize=(10, 20))
sns.heatmap(cluster_profile.T, cmap='rocket_r', annot=False, fmt=".2f")
plt.title('Proporção média das categorias por cluster')
plt.xlabel('Categorias One-Hot')
plt.ylabel('Cluster')
plt.show()

In [ ]:
cluster_profile

In [ ]:
hospitaliz_table = pd.crosstab(df["cluster"], df["HOSPITALIZ"])
hospitaliz_table.columns = ["Sim", "Não", "Ignorado"]
hospitaliz_prop = hospitaliz_table.div(hospitaliz_table.sum(axis=1), axis=0)
hospitaliz_prop = hospitaliz_prop * 100


In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(hospitaliz_prop, annot=hospitaliz_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Hospitalização por Cluster")
plt.xlabel("Tipo de Hospitalização")
plt.ylabel("Cluster")
plt.show()

In [ ]:
evolution_table = pd.crosstab(df["cluster"], df["EVOLUCAO"])
evolution_table.columns = ["cura", "óbito pelo agravo", "óbito por outras causas", "óbito em investigação", "ignorado"]
evolution_prop = evolution_table.div(evolution_table.sum(axis=1), axis=0)
evolution_prop = evolution_prop * 100

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(evolution_prop, annot=evolution_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Evolução por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
classi_table = pd.crosstab(df["cluster"], df["CLASSI_FIN"])
classi_table.columns = ["dengue", "dengue alarmante", "dengue grave"]
classi_prop = classi_table.div(classi_table.sum(axis=1), axis=0)
classi_prop = classi_prop * 100

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(classi_prop, annot=classi_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Tipos de Evolução por Cluster")
plt.xlabel("Tipo de Evolução")
plt.ylabel("Cluster")
plt.show()

In [ ]:
# classi_table = pd.crosstab(df["cluster"], df["CLASSI_FIN"])
# classi_table.columns = ["dengue", "dengue alarmante", "dengue grave"]
# classi_prop = classi_table.div(classi_table.sum(axis=1), axis=0)
# classi_prop = classi_prop * 100

raca_table = pd.crosstab(df["cluster"], df["CS_RACA"])
raca_prop = raca_table.div(raca_table.sum(axis=1), axis=0)
raca_prop.columns = ["branca", "preta", "amarela", "parda", "indigena"]
raca_prop = raca_prop * 100

In [ ]:
plt.figure(figsize=(8, 5))
sns.heatmap(raca_prop, annot=raca_prop.round(2).astype(str) + '%', cmap="OrRd", fmt="")
plt.title("Proporção de Raças por Cluster")
plt.xlabel("Raças")
plt.ylabel("Cluster")
plt.show()

### DBSCAN com One Hot encoding e distância de Hamming

In [ ]:
# from sklearn.cluster import DBSCAN

# sample = df_clustering.sample(frac=0.2, random_state=1)

# db = DBSCAN(eps=0.3, min_samples=10, metric='hamming')
# labels = db.fit_predict(sample)


In [ ]:
# # Supondo que df seja o seu DataFrame binário original
# pca = PCA(n_components=2)
# df_2d = pca.fit_transform(sample)

# # Transforma labels em array numpy
# labels = np.array(labels)

# # Define número de clusters (sem contar outliers)
# n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

# # Cores para os clusters
# plt.figure(figsize=(10, 6))
# unique_labels = set(labels)
# colors = plt.cm.get_cmap("tab10", len(unique_labels))

# for k in unique_labels:
#     class_member_mask = (labels == k)
#     xy = df_2d[class_member_mask]
#     if k == -1:
#         # Cor dos outliers (label -1)
#         plt.scatter(xy[:, 0], xy[:, 1], c='k', marker='x', label='Outliers')
#     else:
#         plt.scatter(xy[:, 0], xy[:, 1], c=[colors(k)], label=f'Cluster {k}')

# plt.legend()
# plt.title(f'DBSCAN com {n_clusters} clusters')
# plt.xlabel('PCA 1')
# plt.ylabel('PCA 2')
# plt.grid(True)
# plt.tight_layout()
# plt.show()

### KMeans - não funciona muito bem para dados categóricos

In [ ]:
# inertias = []
# for k in range(2, 11):
#     # kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(X_scaled)
#     inertias.append(kmeans.inertia_)

# plt.plot(range(2, 11), inertias, marker='o')
# plt.title("Elbow Method")
# plt.xlabel("Número de clusters")
# plt.ylabel("Inércia")
# plt.show()


# kmeans = KMeans(n_clusters=5, random_state=42) 
# labels_kmeans = kmeans.fit_predict(X_scaled)
# df_clustering["cluster"] = labels_kmeans

In [ ]:
# pca = PCA(n_components=3)
# X_pca = pca.fit_transform(X_scaled)

# plt.figure(figsize=(8, 6))
# plt.scatter(X_pca[:, 0], X_pca[:, 1])
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('Visualização dos dados com PCA (2D)')
# plt.grid(True)
# plt.show()

In [ ]:
# 3. Visualizar os dados em 3D
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(
#     X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
#     c='gray',
# )

# ax.set_xlabel("PC1")
# ax.set_ylabel("PC2")
# ax.set_zlabel("PC3")
# ax.set_title("Visualização PCA em 3D")
# plt.show()